In [27]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [28]:
import pandas as pd 
def load_train_data():
    data = pd.read_csv('Acoustic_train.csv')
    return data

def load_test_data():
    data = pd.read_csv('Acoustic_test_no_class.csv')
    return data

train_data = load_train_data()
test_data = load_test_data()

In [29]:
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [30]:
from tensorflow.keras.utils import to_categorical

# One-hot encode 
y_train_encoded = y_train.values
y_train_encoded = pd.factorize(y_train_encoded)[0]  # Convert string labels to numeric
y_train_one_hot = to_categorical(y_train_encoded, num_classes=4)  # Change to the correct number of classes
y_test_one_hot = np.zeros((len(X_test), 4))

In [31]:
model = models.Sequential([
    layers.InputLayer(input_shape=(X_train_scaled.shape[1],)),  # Number of features
    layers.Dense(128, activation='relu'),  # First hidden layer
    layers.Dense(64, activation='relu'),   # Second hidden layer
    layers.Dense(32, activation='relu'),   # Third hidden layer
    layers.Dense(4, activation='softmax')  # Output layer for 4 classes (softmax for multi-class)
])


C:\Users\souri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [32]:
# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',  # For multi-class classification
              metrics=['accuracy'])

# Train the model 
history = model.fit(X_train_scaled, y_train_one_hot, epochs=200, batch_size=32)


Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2625 - loss: 1.4118  
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5858 - loss: 1.1153 
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7297 - loss: 0.9156 
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step - accuracy: 0.7834 - loss: 0.7297
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8212 - loss: 0.5772 
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8536 - loss: 0.4628 
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step - accuracy: 0.8972 - loss: 0.3551
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.9258 - loss: 0.2507
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.9498 - loss: 0.2159
Epoch 10/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.9687 - loss: 0.1721
Epoch 11/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step - accuracy: 0.9803 - loss: 0.1523
Epoch 12/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 81

In [35]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

test_loss, test_acc = model.evaluate(X_test_scaled, y_test_one_hot)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Save the trained model (optional)
model.save('mlp_model_multi_class.h5')



4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2399 - loss: 0.0000e+00  


Test Accuracy: 23.00%


In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy change graph with each epoch
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Predict the classes for the training set
y_train_pred = model.predict(X_train_scaled)
y_train_pred_classes = np.argmax(y_train_pred, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_train_encoded, y_train_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['sad', 'happy', 'relax', 'angry'])

# Plot the confusion matrix
plt.figure(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title('Confusion Matrix')
plt.show()

In [34]:
from tensorflow.keras.models import load_model

# Assuming test_data is available and formatted similarly to train_data
X_test = test_data  # Test data without labels

# Standardize the test data using the same scaler (fit only on training data)
X_test_scaled = scaler.transform(X_test)  # Note: Use transform(), not fit_transform()

# Load the saved model from the .h5 file
model = load_model('mlp_model_multi_class.h5')

# Evaluate the model on the test data
# Since the test set doesn't have labels, we can only perform predictions
y_pred = model.predict(X_test_scaled)

# To get the class predictions, convert the predicted probabilities to class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Print the predicted class labels for the test set
print("Predicted class labels for the test set:")
print(y_pred_classes)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Predicted class labels for the test set:
[2 2 2 0 2 2 3 0 0 1 2 3 0 2 1 0 3 3 0 2 0 2 3 0 2 3 1 0 3 1 3 1 2 2 0 2 1
 3 1 1 3 2 2 2 0 3 1 1 2 1 2 3 1 2 3 2 0 0 2 1 1 0 1 2 1 2 1 0 3 2 2 1 0 0
 3 0 1 3 1 2 2 2 0 2 0 2 0 3 1 2 0 3 1 0 1 3 3 1 1 2]
